# Trabalho final de Metaheurísticas - *problema do caixeiro viajante*

- Disciplina: Meta-heurística
- Prof: Marcos Ribeiro Quinet de Andrade
- Alunos: Bruno Sumar e Cassiano de Souza Santos

# Importando bibliotecas

In [ ]:
from itertools import combinations
from copy import deepcopy
from pprint import pformat
import numpy as np
import sys
import requests
import time

# Declaração de constantes

In [ ]:
# target_url = "https://people.sc.fsu.edu/~jburkardt/datasets/tsp/five_d.txt"
# solution_url = "https://people.sc.fsu.edu/~jburkardt/datasets/tsp/five_s.txt"

# target_url = "https://people.sc.fsu.edu/~jburkardt/datasets/tsp/fri26_d.txt"
# solution_url = "https://people.sc.fsu.edu/~jburkardt/datasets/tsp/fri26_s.txt"

target_url = "https://people.sc.fsu.edu/~jburkardt/datasets/tsp/att48_d.txt"
solution_url = "https://people.sc.fsu.edu/~jburkardt/datasets/tsp/att48_s.txt"

MAX_ITER = 1500
MATRIX_SIZE = 1

# Método de leitura da base de dados

In [ ]:
def read_file_url(url) -> list:
    # Read file with distances between cities from url
    response = requests.get(url)
    data = response.text   
      
    lines = []
    # remove space and tab from file
    for line in data.split("\n"):
        list_with_no_space = []
        for value in line.split(" "):
            if value != "" or value == None:
                list_with_no_space.append(int(float(value)))
        if len(list_with_no_space) > 0:
            lines.append(list_with_no_space)

    return lines

# Métodos auxiliares

In [ ]:
def generate_number() -> int:
    return np.random.randint(0, MATRIX_SIZE, 1)[0]

In [ ]:
def check_cost(combination: list, matrix: list) -> list:
    sum = 0
    for index in range(len(combination)-1):
        sum += matrix[combination[index]][combination[index+1]]
    return sum

In [ ]:
def insertion_cost(matrix: list, i: int, r: int, j: int) -> int:
    return matrix[i][r] + matrix[r][j] - matrix[i][j]

In [ ]:
def swap_items(arr: list, indices: list) -> list:
    arr = arr.copy()
    arr[indices[0]], arr[indices[1]] = arr[indices[1]], arr[indices[0]]
    return arr

In [ ]:
def relink(arr: list, previous: int, value: int) -> list:
    arr = arr.copy()
    arr.remove(value)
    index = arr.index( previous ) + 1 
    arr.insert(index, value)
    return arr

# Métodos construtivos

In [ ]:
def closest_neighbor(matrix: list) -> list:
    matrix = deepcopy(matrix)
    min_value = None
    already_read = []
    sum = 0
    random_number = generate_number()

    for row in range((len(matrix)-1)):

        # Resource to prevent loop
        if len(already_read) != 0:
            for vertex in already_read:
                matrix[random_number][vertex] = 0

        for value in matrix[random_number]:

            if value == 0:
                continue

            else:
                if min_value == None:
                    min_value = int(value)
                else:
                    if value < min_value:
                        min_value = int(value)

        already_read.append(random_number)

        random_number = matrix[random_number].index(min_value)

        sum += min_value

        min_value = None

    sum += max(matrix[already_read[0]])
    already_read.append(random_number)
    already_read.append(already_read[0]) # Return to the first place

    return [already_read, sum]

In [ ]:
def cheapest_insertion(matrix: list) -> list:

    random_number = generate_number()
    solution = [ random_number ]
    
    remaining = [ n for n in range( len(matrix) )]
    remaining.remove( random_number )
    combs = [ c for c in combinations( remaining, 2)]

    best_comb = 0
    best_i_cost = insertion_cost(matrix, random_number, combs[0][0], combs[0][1])
    for index, (r, j) in enumerate( combs[1:] ):
        i_cost = insertion_cost(matrix, random_number, r, j)
        if best_i_cost > i_cost:
            best_comb = index + 1
            best_i_cost = i_cost
            
    solution.append(combs[best_comb][0])
    remaining.remove(combs[best_comb][0])

    solution.append(combs[best_comb][1])
    remaining.remove(combs[best_comb][1])
    
    num_iter = len(remaining)
    for _ in range(num_iter):

        best_x = remaining[0]
        best_pos = 0
        best_i_cost = insertion_cost(matrix, solution[0], best_x, solution[1])
        for x in remaining[1:]:
            
            for pos in range(len(solution)-1)[1:]:
                i_cost = insertion_cost(matrix, solution[pos], x, solution[pos+1])
                
                if best_i_cost > i_cost:
                    best_x = x
                    best_pos = pos
                    best_i_cost = i_cost
                         
        solution.insert(best_pos+1, best_x)
        remaining.remove(best_x)

    solution.append(solution[0])

    return [solution, check_cost(solution, matrix)]

# Métodos de refinamento

In [ ]:
def random_descent(solution: list, cost: int, matrix: list, max_iter: int, _) -> list:

    cur_solution = solution.copy()
    cur_cost = cost

    tam = len( matrix ) - 1
    combs = [ [i+1, j+1] for [i, j] in combinations(range(tam), 2)]
    indices = [ i for i in combs]
    np.random.shuffle(indices)

    for i in range( min(max_iter, len(indices)) ):
        neighbor = swap_items(cur_solution, indices[i])
        
        n_cost = check_cost(neighbor, matrix)

        if n_cost < cur_cost:
            cur_cost = n_cost
            cur_solution = neighbor

    return [cur_solution, cur_cost]

In [ ]:
def path_relinking(solution: list, cost: int, matrix: list, max_iter: int, target_solution: list) -> list:

    if target_solution == None :
        return solution, cost

    target_solution = target_solution.copy()

    cur_solution = solution.copy()
    cur_cost = cost

    best_solution = cur_solution.copy()
    best_cost = cur_cost

    for index in range( len(cur_solution) - 2 ):
        cur_solution = relink(cur_solution[:-1], target_solution[index], target_solution[index+1])
        cur_solution.append( cur_solution[0] )
        cur_cost = check_cost( cur_solution, matrix )

        if cur_cost < best_cost:
            best_solution = cur_solution
            best_cost = cur_cost

    return [best_solution, best_cost]

# GRASP

In [ ]:
def grasp(max_iter: int, construct, refinement) -> list:

    best_solution = None
    best_cost = sys.maxsize

    for i in range(max_iter):
        solution, cost = construct(MATRIX)
        solution, cost = refinement(solution, cost, MATRIX, max_iter, best_solution)

        if cost < best_cost :
            best_cost = cost
            best_solution = solution
        
    return [best_solution, best_cost]

# Carregamento dos dados

In [ ]:
# Loading intercity distance matrix
MATRIX = read_file_url( target_url )
MATRIX_SIZE = len( MATRIX )
print("Reading set of %d cities" % ( MATRIX_SIZE ))

# Loading itinerary that minimizes the total distance
SOLUTION = [ n-1 for n in np.array( read_file_url(solution_url) ).transpose()[0] ]
if len( SOLUTION ) < MATRIX_SIZE + 1 :
    SOLUTION.append( SOLUTION[0] ) 
SOLUTION_COST = check_cost( SOLUTION, MATRIX ) 

Reading set of 48 cities


# Solução

In [ ]:
print(">> Itinerary that minimizes the total distance:\n")
print("    Total distance: %s" % ( SOLUTION_COST ))
print("    Itinerary:      ", end = '')
formatted = pformat(SOLUTION, width=60, compact=True, indent=20)
formatted = formatted if len(formatted) <= 60 else ''.join( swap_items([c for c in formatted], [0,19])[19:] )
print(formatted)

>> Itinerary that minimizes the total distance:

    Total distance: 33551
    Itinerary:      [0, 7, 37, 30, 43, 17, 6, 27, 5, 36, 18,
                    26, 16, 42, 29, 35, 45, 32, 19, 46, 20,
                    31, 38, 47, 4, 41, 23, 9, 44, 34, 3, 25,
                    1, 28, 33, 40, 15, 21, 2, 22, 13, 24,
                    12, 10, 11, 14, 39, 8, 0]


# Execução do método GRASP

In [ ]:
constructors = [ closest_neighbor, cheapest_insertion ]
refinements = [ random_descent, path_relinking]

for c in constructors :
    for r in refinements :
        print("\n>>  Using %s and %s:\n" % (c.__name__, r.__name__))
        start_time = time.time()
        result = grasp(MAX_ITER, c, r)
        print("    Total distance: %s" % (result[1]))
        print("    Error:          %.1f%%" % ( 100 * (result[1] - SOLUTION_COST) / SOLUTION_COST ))
        print("    Exceution time: %.3f seconds" % (time.time() - start_time))
        print("    Itinerary:      ", end = '')
        formatted = pformat(result[0], width=60, compact=True, indent=20)
        formatted = formatted if len(formatted) < 60 else ''.join( swap_items([c for c in formatted], [0,19])[19:] )
        print(formatted, "")


>>  Using closest_neighbor and random_descent:

    Total distance: 35634
    Error:          6.2%
    Exceution time: 15.945 seconds
    Itinerary:      [34, 44, 9, 23, 41, 4, 47, 38, 31, 20,
                    46, 10, 22, 13, 24, 12, 39, 14, 11, 19,
                    32, 45, 43, 6, 35, 27, 5, 36, 18, 16,
                    42, 26, 29, 17, 30, 37, 8, 0, 7, 21, 15,
                    2, 33, 40, 28, 1, 25, 3, 34] 

>>  Using closest_neighbor and path_relinking:

    Total distance: 37704
    Error:          12.4%
    Exceution time: 3.436 seconds
    Itinerary:      [22, 10, 11, 19, 32, 45, 14, 39, 18, 36,
                    5, 29, 26, 42, 16, 27, 6, 17, 35, 43,
                    30, 37, 8, 0, 7, 21, 15, 2, 33, 40, 41,
                    9, 23, 44, 34, 25, 3, 1, 28, 4, 47, 38,
                    31, 20, 46, 12, 24, 13, 22] 

>>  Using cheapest_insertion and random_descent:

    Total distance: 34934
    Error:          4.1%
    Exceution time: 23.713 seconds
    Itinerary:   

# Relatório

## Métodos utilizados
### Construtivos
1. **Vizinho mais próximo** - Primeiramente uma cidade é escolhida aleatoriamente para ser o ponto de partida, em seguida é feita uma busca pela cidade vizinha que tenha o menor custo de distância, repetindo esse processo até que todas as cidades tenham sido visitadas.

1. **Inserção mais barata** - Uma cidade é escolhida aleatoriamente, em seguida duas cidades são escolhidas de forma a minimizar o custo de inserção da primeira cidade, depois é feita uma análise dos custos das possíveis inserções das cidades restante, escolhendo o melhor resultado para fazer parte da solução, repetindo esse processo até que todas as cidades tenham sido visitadas.

### Refinamento
1. **Descida aleatória** (Busca Local) - Com a solução gerada pelo método construtivo, é realizada uma série de pequenas alterações verificando se houve uma melhora do resultado.

1. **Reconexão de caminhos** - Partindo de duas soluções, a primeira é alterada pouco a pouco, utilizando a segunda solução como uma guia, verificando se houve uma melhora em cada estágio. 

## Resultados

Analisando todas as combinações de métodos construtivos com os de refinamento, utilizando como base de dados [ATT48](https://people.sc.fsu.edu/~jburkardt/datasets/tsp/att48_d.txt)  o melhor resultado gerado foi utilizando os métodos de inserção mais baratada e descida aleatória, com uma média de 6% de erro e 24 segundos para a conclusão do algoritmo.